In [5]:
import os
import pandas as pd
import tensorflow
import numpy as np
import matplotlib
import os
import re
import sentencepiece as spm
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

## LOAD DATA

In [6]:
df = pd.read_csv('data2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class         3950 non-null   object
 1   conversation  3950 non-null   object
 2   okt           3950 non-null   object
 3   mecab         3950 non-null   object
dtypes: object(4)
memory usage: 123.6+ KB


In [7]:
df['conversation'][4]

'저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에 발라보실래요 아 진짜요 안 그래도 선크림 필요해서 알아보던 중인데 한 번 발라 볼게요 여기 한 번 발라보세요 진짜 성분도 좋고 다 좋아요 음 성분이 좋다고 하셔서 좋은거 같기는 한데 제 피부에 맞지 않나봐요 피부가 따끔거리네요 이번에 진짜 열심히 연구해서 만든건데 피부가 많이 예민하신가봐요 네 많이 예민해요 그럼 많이 파시고 안녕히 계세요 아니 저기요 돈 안내요 네 발라보는것도 돈 내야 하나요 그럼 이거 누구한테 팔아요 당신이 바른거를 아니 먼저 발라 보시라고 하셨잖아요 먼저 권유해놓고 사라고 강매하는거 갈취인거 몰라요 내가 안 사도 된다고 말 한 적 있어 그것도 모르고 바른걸 누구 탓 하나 빨리 사 당신이 바른거 당신이 사야지 진짜 어이가 없어서 다른 사람들한텐 이렇게 갈취하지마세요 화딱지나네'

In [8]:
df.head()

,class,conversation,okt,mecab
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자...,"['지금', '너', '스스로', '를', '죽여', '달라', '고', '애원',...","['지금', '너', '스스로', '를', '죽여', '달', '라고', '애원',..."
1,협박 대화,길동경찰서입니다 시 분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만 더 얘기한...,"['길동', '경찰서', '입니다', '시', '분', '마트', '에', '폭발물...","['길동', '경찰서', '입니다', '시', '분', '마트', '에', '폭발물..."
2,기타 괴롭힘 대화,너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지...,"['너', '되게', '귀여운거', '알', '지', '나', '보다', '작은',...","['너', '되게', '귀여운', '거', '알', '지', '나', '보다', '..."
3,갈취 대화,어이 거기 예 너 말이야 너 이리 오라고 무슨 일 너 옷 좋아보인다 얘 돈 좀 있나...,"['어이', '거기', '예', '너', '말', '이야', '너', '이리', '...","['어', '이', '거기', '예', '너', '말', '이', '야', '너',..."
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에 발...,"['저기', '요', '혹시', '날', '이', '너무', '뜨겁잖아요', '저희...","['저기', '요', '혹시', '날', '이', '너무', '뜨겁', '잖아요',..."


In [9]:
# 레이블 인코딩
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['class'])

### 패딩, 토크나이징

In [10]:
# 텍스트 토큰화 및 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['conversation'])
sequences = tokenizer.texts_to_sequences(df['conversation'])
max_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_length, padding='pre')
y = df['label']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# LSTM 모델 구성 함수
# LSTM 모델 구성 함수
def create_lstm_model(embedding_dim, lstm_units, hidden_units, dropout_rate=0.4):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=max_length))
    model.add(LSTM(units=lstm_units, dropout=dropout_rate))
    model.add(Dense(hidden_units, activation='relu'))  # 추가된 은닉 레이어
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# KerasClassifier를 활용하여 Keras 모델을 scikit-learn 모델로 래핑
model = KerasClassifier(build_fn=create_lstm_model, verbose=0)

# 그리드 서치를 통한 모델 하이퍼파라미터 튜닝
param_grid = {
    'embedding_dim': [50, 100],
    'lstm_units': [64, 128],
    'hidden_units':[64, 128, 256],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_result = grid_search.fit(X_train, y_train)

# 최적 모델의 하이퍼파라미터 출력
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Fitting 3 folds for each of 12 candidates, totalling 36 fits


In [ ]:
import tensorflow as tf

# 클래스 인덱스를 원-핫 인코딩 형태로 변환
y_train_encoded = tf.one_hot(y_train, depth=4)  # 클래스 개수인 4로 depth 설정
y_test_encoded = tf.one_hot(y_test, depth=4)


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5', save_best_only=True),
    LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** epoch))
]

# 모델 학습
best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train_encoded, epochs=20, batch_size=32, validation_data=(X_test, y_test_encoded), callbacks=callbacks)


In [ ]:
# 테스트 데이터로 평가
y_pred = best_model.predict(X_test)
y_pred = np.round(y_pred).flatten()
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
# 모델 내부 Keras 모델 저장
best_model.model.save('best_lstm_model.h5')


In [ ]:
test = pd.read_json('./data/test.json').T
test.head()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test['text'])
sequences = tokenizer.texts_to_sequences(test['text'])
max_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_length, padding='pre')

In [ ]:
from keras.models import load_model
import numpy as np

# 저장한 모델 로드
loaded_model = load_model('best_lstm_model.h5')

# 테스트 데이터에 대한 예측 수행
y_pred_probs = loaded_model.predict(X)

# 예측 확률을 클래스 인덱스로 변환
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# 예측된 클래스 출력
print(y_pred_classes)


In [ ]:
# 라벨 디코딩을 위한 역변환
decoded_classes = label_encoder.inverse_transform(y_pred_classes)

# 디코딩된 클래스 출력
print(decoded_classes)


In [ ]:
test['class'] = decoded_classes


In [ ]:
test.head

In [ ]:
# 디코딩된 클래스를 새로운 인코딩 값으로 매핑하는 딕셔너리 생성
new_encoding_dict = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
}

# 디코딩된 클래스를 새로운 인코딩 값으로 변환
new_encoded_classes = [new_encoding_dict[decoded] for decoded in decoded_classes]

# 변환된 새로운 인코딩 값 출력
print(new_encoded_classes)


In [ ]:
test['class'] = new_encoded_classes

In [ ]:
test

In [ ]:
submission = test[['class']]
submission

In [ ]:
submission.to_csv('try1.csv')

In [ ]:
test['class'] = 0
test

In [ ]:
submission = test[['class']]


In [ ]:
submission.to_csv('try2.csv')

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the create_lstm_model function
def create_lstm_model(embedding_dim, lstm_units, hidden_units, dropout_rate=0.4):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=max_length))
    model.add(LSTM(units=lstm_units, dropout=dropout_rate))
    model.add(Dense(hidden_units, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a model instance
model = create_lstm_model(embedding_dim=100, lstm_units=128, hidden_units=128)

# Plot the model and save it as an image file
plot_model(model, to_file='lstm_model.png', show_shapes=True, show_layer_names=True)
